### <font color='red'>Quora Question Pairs - Combined LSTM Model</font>

#### Import required libraries

In [2]:
import pandas as pd
import itertools as itertools
import sklearn as skl
import numpy as np
import matplotlib.pyplot as plt
import nltk as nk

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer, confusion_matrix

from keras.models import Sequential, Model
from keras.layers import Input, Embedding, LSTM, Merge, Dense, Dropout, concatenate
from keras.optimizers import Adadelta
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

from nltk.corpus import stopwords

import re
import Levenshtein as leven
from gensim.models import KeyedVectors
from math import sqrt

Using Theano backend.
C:\Users\lim_j\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


#### Import datasets and clean data, for practice.
The training dataset provided will be split into train-test to validate model's accuracy

In [3]:
train = pd.read_csv(r'C:\Users\lim_j\Google Drive\Technical Skills\Kaggle\Quora Question Pairs\train.csv')
test = pd.read_csv(r'C:\Users\lim_j\Google Drive\Technical Skills\Kaggle\Quora Question Pairs\test.csv')
embedding_file = r'C:\Users\lim_j\Google Drive\Technical Skills\Kaggle\Quora Question Pairs\GoogleNews-vectors-negative300.bin.gz'
model_dir = r'C:\Users\lim_j\Google Drive\Technical Skills\Kaggle\Quora Question Pairs\Model'

## Exploratory Data Analysis (EDA)

In [4]:
train.describe()

,id,qid1,qid2,is_duplicate
count,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198
std,116708.614502,157751.700002,159903.182629,0.482588
min,0.000000,1.000000,2.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000
50%,202144.500000,192182.000000,197052.000000,0.000000
75%,303216.750000,346573.500000,354692.500000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404288 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


There are 2 null values under column question2, hence we will have to remove these 2 pairs of questions with null values.

In [6]:
type(train.question2[0])

str

In [7]:
train[train.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0


In [8]:
train = train.dropna(axis=0, how='any')

In [9]:
# Verify that rows with null values have been removed
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404288 entries, 0 to 404289
Data columns (total 6 columns):
id              404288 non-null int64
qid1            404288 non-null int64
qid2            404288 non-null int64
question1       404288 non-null object
question2       404288 non-null object
is_duplicate    404288 non-null int64
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


## Data Munging
Derived from Elior Cohen's data cleaning process

In [10]:
def text2word(text):
    text = str(text).lower()

    # Text cleaning
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = text.split()
    return text


# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # placeholder for the [0, 0, ....0] embedding / padding
word2vec = KeyedVectors.load_word2vec_format(embedding_file, binary=True)
stops = set(stopwords.words("english"))

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [train, test]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:
            q2n = []  # q2n -> question numbers representation
            
            for word in text2word(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                    
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

### Train-Validation data split for x and y values and zero padding of values to ensure consistency in shape of data

In [11]:
# To obtain the max length of the longest question in train/test datasets
max_seq_length = max(train.question1.map(lambda x: len(x)).max(),
                     train.question2.map(lambda x: len(x)).max(),
                     test.question1.map(lambda x: len(x)).max(),
                     test.question2.map(lambda x: len(x)).max())

# Split to train validation (80-20 split)
validation_size = 0.2
training_size = len(train)*(1-validation_size)

# Breaking dataframe values into x (question strings) and y (is_duplicate = 1/0)
x = train[questions_cols]
y = train['is_duplicate']

# Split data using train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=validation_size)

# Split to dictionaries for training, validation and test data
x_train = {'left': x_train.question1, 'right': x_train.question2}
x_validation = {'left': x_validation.question1, 'right': x_validation.question2}
x_test = {'left': test.question1, 'right': test.question2}

# Convert y-values (labels) to their numpy representations
y_train = y_train.values
y_validation = y_validation.values

# Zero padding
for dataset, side in itertools.product([x_train, x_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Checking consistent shapes for data using assert
assert x_train['left'].shape == x_train['right'].shape
assert len(x_train['left']) == len(y_train)

### Model building and training

In [12]:
# Model hyperparameters
hidden_layer_nodes = 50
batch_size = 64
epochs = 1

# declare left and right inputs
left_input = Input(shape=(max_seq_length, ), dtype='int32')
right_input = Input(shape=(max_seq_length, ), dtype='int32')

# Create an embedding layer to convert words to their embeddings
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Convert inputs into word embeddings
embedded_left = embedding_layer(left_input)
embedded_right = embedding_layer(right_input)

# lstm layer that will return an output the size of the number of hidden layer nodes
shared_lstm = LSTM(hidden_layer_nodes)

# run both inputs through shared lstm
encoded_left = shared_lstm(embedded_left)
encoded_right = shared_lstm(embedded_right)

# concatenate results of both encoded vectors
merged_vector = concatenate([encoded_left, encoded_right], axis=-1)

# finish off model with output layer
prediction = Dense(1, activation='relu')(merged_vector)

# Define model hyperparameters such as optimiser and loss function
model = Model(inputs=[left_input, right_input], outputs=[prediction])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# summarize the model
print(model.summary())

# Start training
model_trained = model.fit([x_train['left'], x_train['right']], y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=([x_validation['left'], x_validation['right']], y_validation), verbose=1)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 213)           0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 213)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 213, 300)      36427500    input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 50)            70200       embedding_1[0][0]       

### Evaluate model scalar test loss and accuracy

In [13]:
test_loss_accuracy = model.evaluate([x_validation['left'], x_validation['right']], y_validation)
print('Scalar test loss: ' + str(test_loss_accuracy[0]) + '\nModel accuracy: ' + str(test_loss_accuracy[1]))

80858/80858 [==============================] - 209s   
Scalar test loss: 0.176279500335
Model accuracy: 0.740928541398
